# Install yival😺

In [ ]:
%pip install openai
%pip install yival
%pip install numpy
import openai
import os

# Make sure you have config your openai_api_key

In [ ]:
def openai_completion(message:str, tem=0.3) -> str:
  openai.api_key = os.getenv("OPENAI_API_KEY")
  messages = [
    {
        "role":"user",
        "content":message
    }
  ]

  response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=tem
    )
  res = response['choices'][0]['message']['content']
  return res

os.environ['OPENAI_API_KEY']=''
print(openai_completion('hello chatgpt!'))

# Pipeline prompt Generate
1. Generate prompt with base_prompt or improve_prompt
2. view prompt output with test_variations
3. human give suggestions , then goto 1

1->2->3->1->...

Loop until you have find your best prompt

## Set prompt config and test_variations

"In the current demo, we want GPT-4 to generate a prompt for 'Writing a landing page headline based on startup company information'

And we use two company name for test : 'BrightPath Innovations' and 'SparkleEdge Technologies'

In [1]:
from yival.wrappers.string_wrapper import StringWrapper
import pandas as pd

base_prompt = """
Hi, You are the best prompt writer!
Your object is to construct a concise instruction prompt for GPT-4. This prompt will guide GPT-4 in its interactions as a professional writer who writes attractive landing page headline for startup business according to its name.
Points to emphasize in your instruction:
 - GPT4 will play the role of an experienced, mature and sophisticated writer who is familiar with all kinds of pop-up landing page headlines and is very creative.
 - The GPT-4 response should conform to the basic structure of the company's landing page headline, be able to fully reflect the company's identity, and be able to attract a sufficiently large number of people
 - {company_name} represent the name of the company
 - At all times, GPT-4 must give a response as landing page headline.

Craft your instruction ensuring GPT-4 understands the name of the startup company and generate attractive professional landing page headline according to its name.
keep your output crisp: only the prompt, devoid of any extraneous content.
"""

improve_prompt = """
Hi , you are the best prompt improver!
Your object is to improve the given prompt according to the suggestions.

Understand the given prompt and the suggestion , correct the prompt according to the suggestion , and give prompt back.
Remember your goal is to write prompt according to the given prompt , the goal of the prompt should be same , just improve it.

- prompt:{prompt}
- suggestion:{suggestion}
keep your output crisp: only the prompt, devoid of any extraneous content.
"""

gen_prompts = [base_prompt]
df_result = []

In [2]:
# design your custom_func
def custom_func(prompt:str,variation:str) -> str:
  return openai_completion(str(StringWrapper(prompt,"",variation)),tem=0.7)

In [ ]:
def optimize_prompt(base_prompt:str, suggestions:str)->str:
  if suggestions:
    generate_prompt = openai_completion(str(StringWrapper(improve_prompt,"",{"prompt":base_prompt,"suggestion":suggestions})),tem=0.4)
  else:
    generate_prompt = openai_completion(base_prompt)
  gen_prompts.append(generate_prompt)
  return generate_prompt


def view_output(prompt:str, example_variations:list)->None:
  result = [prompt] + [custom_func(prompt,var) for var in example_variations]
  # print(f"result:{result}")
  # for var_name,gen in result:
  #   print(f"{var_name}: {gen}")
  # df_result = [r[0] for r in result]
  df_result.append(result)

In [ ]:
# test variations
test_variations = [{'company_name': 'BrightPath Innovations'},{'company_name': 'SparkleEdge Technologies'}]

# Set suggestions = None for first generate
suggestions = None

## Step1: Generate prompt with base_prompt or improve_prompt

In [ ]:
#Step 1: generate new prompt
new_prompt = optimize_prompt(gen_prompts[-1],suggestions)
print(f"New prompt is now : {new_prompt}")

## Step2 : View output of the new result

In [ ]:
#Step 2: view output of the new result
view_output(new_prompt, test_variations)
pd.DataFrame(df_result,columns= ["prompt"] + [str(v) for v in test_variations] )

## Step 3: Write suggestions to the prompt according to the variation outputs

In [3]:
#Step3: Write suggestions to the prompt according to the variation outputs
suggestions = "I want the headline to be more creative!"
improver_prompt = StringWrapper(improve_prompt,"",{"prompt":gen_prompts[-1],"suggestion":suggestions})
print(f"improver prompt: \n {improver_prompt}")

improver prompt: 
 
Hi , you are the best prompt improver!
Your object is to improve the given prompt according to the suggestions.

Understand the given prompt and the suggestion , correct the prompt according to the suggestion , and give prompt back.
Remember your goal is to write prompt according to the given prompt , the goal of the prompt should be same , just improve it.

- prompt:
Hi, You are the best prompt writer!
Your object is to construct a concise instruction prompt for GPT-4. This prompt will guide GPT-4 in its interactions as a professional writer who writes attractive landing page headline for startup business according to its name.
Points to emphasize in your instruction:
 - GPT4 will play the role of an experienced, mature and sophisticated writer who is familiar with all kinds of pop-up landing page headlines and is very creative.
 - The GPT-4 response should conform to the basic structure of the company's landing page headline, be able to fully reflect the company's

## if the prompt is not good enough : Goto Step1